In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

# Cargar archivo 
df = pd.read_csv("../../data/raw/Ibague-co.csv")

# Renombrar columnas
df.rename(columns={
    "date": "Date", 
    "tmax": "Tmax", 
    "tmin": "Tmin", 
    "tavg": "Tavg",
    "prcp": "Prcp", 
    "wspd": "Wspd", 
    "wdir": "Wdir"
}, inplace=True)

# Convertir columnas numéricas
for col in ["Tmax", "Tmin", "Tavg", "Prcp", "Wspd"]:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    media_redondeada = round(df[col].mean(), 1)
    df[col].fillna(media_redondeada,)
    df[col] = df[col].round(1)

# Convertir fecha
df["Date"] = pd.to_datetime(df["Date"]).dt.date

# Convertir Wspd a km/h
df["Wspd"] = (df["Wspd"] * 3.6).round(1)

# Asignar dirección de viento según fecha
def direccion_viento(fecha):
    mes_dia = (fecha.month, fecha.day)
    if (2, 12) <= mes_dia <= (2, 19) or (10, 16) <= mes_dia <= (11, 5):
        return "Oeste"
    elif (2, 19) <= mes_dia <= (10, 16):
        return "Sur"
    else:
        return "Norte"

df["Wdir"] = df["Date"].apply(direccion_viento)

# Agregar columnas fijas
df["Estacion"] = "Ibague"
df["Provincia"] = "Tolima"
df["Pais"] = "Colombia"

# Eliminar columnas irrelevantes si existen
df.drop(columns=["snow", "wpgt", "pres", "tsun"], inplace=True, errors='ignore')

# Reordenar columnas
df = df[["Date", "Tmax", "Tmin", "Tavg", "Prcp", "Wspd", "Wdir", "Estacion", "Provincia", "Pais"]]

# Guardar
salida = "../../data/cleaned/datos_limpios_ibague_2024.csv"
os.makedirs(os.path.dirname(salida), exist_ok=True)
df.to_csv(salida, index=False, encoding="utf-8")

print(f"Archivo limpio guardado en: {salida}")